In [1]:
import seaborn
import pandas as pd
import wandb
from pandas_profiling import ProfileReport
import numpy as np

Create wandb run & get artifact

In [2]:
run = wandb.init(
  project="exercise_4",
  save_code=True
)

wandb: Currently logged in as: charly-ko (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
artifact = run.use_artifact("exercise_4/genres_mod.parquet:latest")
local_path = artifact.file()
print(local_path)
df = pd.read_parquet(local_path)
df.head()

./artifacts/genres_mod.parquet:v0/genres_mod.parquet


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,genre,song_name,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,audio_features,124539,4,Dark Trap,Mercury: Retrograde,None
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,audio_features,224427,4,Dark Trap,Pathology,None
2,0.850,0.893,5,NaN,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,audio_features,98821,4,Dark Trap,Symbiote,None
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,audio_features,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),None
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,audio_features,123298,4,Dark Trap,Venom,None


Create pandas profiling report

In [6]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

/home/carlotta/miniconda3/envs/mlflow-403062032ff60d937999a3b34dff672043d333cc/lib/python3.8/site-packages/pandas_profiling/model/summary.py:207: UserWarning: There was an attempt to generate the matrix missing values diagrams, but this failed.
    To hide this warning, disable the calculation
    (using `df.profile_report(missing_diagrams={"matrix": False}`)
    If this is problematic for your use case, please report this as an issue:
    https://github.com/pandas-profiling/pandas-profiling/issues
    (include the error message: 'keyword grid_b is not recognized; valid keywords are ['size', 'width', 'color', 'tickdir', 'pad', 'labelsize', 'labelcolor', 'zorder', 'gridOn', 'tick1On', 'tick2On', 'label1On', 'label2On', 'length', 'direction', 'left', 'bottom', 'right', 'top', 'labelleft', 'labelbottom', 'labelright', 'labeltop', 'labelrotation', 'grid_agg_filter', 'grid_alpha', 'grid_animated', 'grid_antialiased', 'grid_clip_box', 'grid_clip_on', 'grid_clip_path', 'grid_color', 'grid_das

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Auffaellingkeiten: 
- type kann wohl gelöscht werden --> immer konstant
- 921 Duplikate --> können gelöscht werden 
- Was hat es mit tile aus sich? und wie ist das Verhältnis zu songname?
- Type, mode, time_signature, gerne scheinen sehr ähnlich zu sein 
- sowohl bei songname als auch bei Titel fehlen sehr viele Einträge
...

Duplikate entfernen

In [6]:
df = df.drop_duplicates().reset_index(drop=True)

Feature Engineering: Zusammenbring von title und song_name
Das Feature sollte dann in den Feature Story übergeben werden --> wenn nicht, dann sollte es in der Pipeline berechnet werden

In [7]:
df['title'].fillna(value=' ', inplace=True)
df['song_name'].fillna(value= ' ', inplace=True)
df['text_feature'] = df['title'] + ' ' + df['song_name']

In [8]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_widgets()

Summarize dataset:   0%|          | 0/31 [00:00<?, ?it/s]

/home/carlotta/miniconda3/envs/mlflow-403062032ff60d937999a3b34dff672043d333cc/lib/python3.8/site-packages/pandas_profiling/model/summary.py:207: UserWarning: There was an attempt to generate the matrix missing values diagrams, but this failed.
    To hide this warning, disable the calculation
    (using `df.profile_report(missing_diagrams={"matrix": False}`)
    If this is problematic for your use case, please report this as an issue:
    https://github.com/pandas-profiling/pandas-profiling/issues
    (include the error message: 'keyword grid_b is not recognized; valid keywords are ['size', 'width', 'color', 'tickdir', 'pad', 'labelsize', 'labelcolor', 'zorder', 'gridOn', 'tick1On', 'tick2On', 'label1On', 'label2On', 'length', 'direction', 'left', 'bottom', 'right', 'top', 'labelleft', 'labelbottom', 'labelright', 'labeltop', 'labelrotation', 'grid_agg_filter', 'grid_alpha', 'grid_animated', 'grid_antialiased', 'grid_clip_box', 'grid_clip_on', 'grid_clip_path', 'grid_color', 'grid_das

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
run.finish()